In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, average_precision_score, precision_score, recall_score
import pickle
import tqdm
import PIL
from PIL import Image

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.python.keras.applications import VGG19, ResNet50, Xception
from tensorflow.python.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import Adam, RMSprop
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.python.keras.layers import concatenate

In [3]:
def print_layer_trainable():
    for layer in model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

In [23]:
with open('healthyVGG19First.pickle', 'rb') as handle:
    healthyData = pickle.load(handle)

In [24]:
ids = [x for x in os.listdir('./Data/')]

labels = {}

for x in ids:
    y = x.replace('.jpg', '').rstrip('0123456789')
    labels[x] = healthyData.loc[y].tolist()
    
X, Y = train_test_split(ids, test_size=0.2, shuffle=True)

In [114]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [31]:
save_obj(labels, 'labels')
save_obj(X, 'train')
save_obj(Y, 'test')

In [4]:
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
labels = load_obj('./Pickles/labelsReal')
labels2 = load_obj('./Pickles/labelsCat')
X = load_obj('./Pickles/train')
Y = load_obj('./Pickles/test')

In [5]:
partition = {}
partition['train'] = X
partition['test'] = Y

In [39]:
model2 = VGG19(include_top=True, weights='imagenet')

In [40]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [38]:
model.layers[0].output_shape

(None, 224, 224, 3)

In [10]:
def generator(indices, batchSize = 50):
    while True:
        np.random.shuffle(indices)
        
        for i in range(0,50, batchSize):
            X = []
            Y = []
            batch_indices = indices[i:i+batchSize]
            batch_indices.sort()
            print(batch_indices)
            
            for x in batch_indices:
                X.append(np.array(Image.open('./Data/' + x))/255)
                Y.append(labels[x])
        
            X = np.array(X)
            Y = np.array(Y)
            yield (X, Y)
            break
        break

In [79]:
indices = partition['train']
batchSize = 10

In [38]:
batchSize = 10
training=True
indices = partition['test']
while True:
    if training:
        np.random.shuffle(indices)

    for i in range(0,len(indices), batchSize):
        X = []
        real = []
        build = []
        eye = []
        hair = []
        race = []
        batch_indices = indices[i:i+batchSize]
        if training:
            batch_indices.sort()
        print(batch_indices)

        for x in batch_indices:
            X.append(np.array(Image.open('./Data2/' + x))/255)
            real.append(labels[x])
            build.append(labels2[x][:4])
            eye.append(labels2[x][4:10])
            hair.append(labels2[x][10:16])
            race.append(labels2[x][16:])

        X = np.array(X)
        real = np.array(real)
        build = np.array(build)
        eye = np.array(eye)
        hair = np.array(hair)
        race = np.array(race)
        #yield (X, {'real':real, 'build':build, 'eye':eye, 'hair':hair, 'race':race})
    break

['chanel iman39.jpg', 'charlotte riley249.jpg', 'chris evans225.jpg', 'janel parrish92.jpg', 'janelle monae183.jpg', 'jennifer love hewitt143.jpg', 'paul rudd18.jpg', 'ray stevenson202.jpg', 'sushmita sen29.jpg', 'toni garrn366.jpg']
['aaron taylor johnson217.jpg', 'alicia vikander297.jpg', 'beyonce knowles445.jpg', 'cristiano ronaldo259.jpg', 'emilia clarke145.jpg', 'hilary swank131.jpg', 'julie benz256.jpg', 'kelly osbourne187.jpg', 'lionel messi128.jpg', 'quentin tarantino355.jpg']
['bethenny frankel317.jpg', 'brenda song215.jpg', 'brendon urie212.jpg', 'caroline trentini116.jpg', 'denzel washington465.jpg', 'ian somerhalder417.jpg', 'jessica szohr162.jpg', 'kelly osbourne144.jpg', 'michelle pfeiffer131.jpg', 'roberto firmino164.jpg']
['angelique kerber193.jpg', 'blac chyna101.jpg', 'dinah jane hansen371.jpg', 'madison keys290.jpg', 'padma lakshmi298.jpg', 'ruth negga174.jpg', 'stephen amell408.jpg', 'troian bellisario173.jpg', 'uzo aduba35.jpg', 'zinedine zidane285.jpg']
['andre ig

KeyboardInterrupt: 

In [26]:
generator_train = generator(partition['train'])

In [27]:
generator_test = generator(partition['test'])

In [41]:
transfer_layer = model.get_layer('avg_pool')

In [16]:
transfer_layer

NameError: name 'transfer_layer' is not defined

In [43]:
conv_model = Model(inputs=model.input, outputs=transfer_layer.output)

In [48]:
# Start a new Keras Sequential model.
new_model = Sequential()

# Add the convolutional part of the VGG19 model from abo

new_model.add(Flatten())
new_model.add(Dense(4096, activation='relu', name='hi'))
new_model.add(Dropout(0.5))
new_model.add(Dense(4096, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(25, activation='sigmoid'))

In [49]:
new_model.layers[1].name = 'test'

AttributeError: can't set attribute

# Models

In [145]:
from __future__ import print_function

import numpy as np
import warnings

from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Input
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import MaxPooling2D
from tensorflow.python.keras.layers import GlobalMaxPooling2D
from tensorflow.python.keras.layers import GlobalAveragePooling2D

In [118]:
model = VGG19(include_top=False, weights='imagenet', input_shape = (224,224, 3))
#model2 = VGG19_2(include_top=False, weights='imagenet', input_shape = (224,224, 3))

In [119]:
model.save_weights("./Weights/basic.h5")

In [151]:
img_input = Input(shape=(224,224, 3))
# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1_2')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool_2')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1_2')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool_2')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1_2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2_2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3_2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool_2')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1_2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2_2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3_2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool_2')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1_2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2_2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3_2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool_2')(x)

inputs = img_input
# Create model.
model2 = Model(inputs, x, name='model2')
model2.load_weights('./Weights/basic.h5')

In [152]:
for layer in model.layers:
    layer.trainable=False

for layer in model2.layers:
    layer.trainable=False

In [153]:
concatenatedOut = concatenate([model.output, model2.output], axis=-1)

In [154]:
x = Flatten()(concatenatedOut)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
prediction1 = Dense(3, activation = 'sigmoid', name='real')(x)
prediction2 = Dense(4, activation = 'softmax', name='build')(x)
prediction3 = Dense(6, activation = 'softmax', name='eye')(x)
prediction4 = Dense(6, activation = 'softmax', name='hair')(x)
prediction5 = Dense(5, activation = 'softmax', name='race')(x)

In [155]:
finalModel = Model(inputs=[model.input, model2.input], outputs=[prediction1, prediction2, prediction3, prediction4, prediction5])

In [157]:
finalModel.compile(optimizer=Adam(lr=1e-5), loss=
                      {'real': 'mean_squared_error', 'build': 'categorical_crossentropy',
                      'eye': 'categorical_crossentropy', 'hair': 'categorical_crossentropy',
                      'race': 'categorical_crossentropy'}, metrics=['mse', 'acc'])

In [158]:
finalModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_13[0][0]                   
__________________________________________________________________________________________________
block1_conv1_2 (Conv2D)         (None, 224, 224, 64) 1792        input_15[0][0]                   
__________________________________________________________________________________________________
block1_con

In [159]:
# serialize model to JSON
model_json = finalModel.to_json()
with open("./Architecture/VGG2Input.json", "w") as json_file:
    json_file.write(model_json)

In [160]:
# serialize weights to HDF5
finalModel.save_weights("./Weights/VGG2Input.h5")
print("Saved model to disk")
#loaded_model.compile()

Saved model to disk


In [8]:
from tensorflow.python.keras import backend as K
K.clear_session()

In [5]:
# load json and create model
optimizer = Adam(lr=1e-5)
loss = {'real': 'mean_squared_error', 'build': 'categorical_crossentropy',
                  'eye': 'categorical_crossentropy', 'hair': 'categorical_crossentropy',
                  'race': 'categorical_crossentropy'}
metrics = ['mse', 'accuracy']

json_file = open('./Architecture/VGG.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./Weights/VGG5/weights-07-5.22.hdf5")
print("Loaded model from disk")
loaded_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Loaded model from disk


In [6]:
image = np.array(Image.open('./Data2/iggy azalea129.jpg'))
image = np.expand_dims(image, axis=0)

In [7]:
preds = loaded_model.predict(image)

In [63]:
prob = []

for i in range(1, 5):
    b = np.zeros_like(preds[i][0])
    b[preds[i][0].argmax()] = 1
    prob = np.concatenate((prob, b), axis=0)
    
prob = np.expand_dims(prob, axis=0)

In [94]:
genderDict = load_obj('genderDict')

In [105]:
del genderDict[[x for x in genderDict.keys() if x not in healthyData.index.tolist()][0]]

In [108]:
genderDict['drew barrymore'] = 'female'

In [109]:
len(genderDict)

1366

In [17]:
with open('./Pickles/healthyVGG19Final.pickle', 'rb') as handle:
    healthyData = pickle.load(handle)

In [110]:
gender = {'male':[],'female':[]}

for key, value in genderDict.items():
    if value == 'male':
        gender['male'].append(key)
    else:
        gender['female'].append(key)
    
gender['male'].sort()
gender['female'].sort()

In [115]:
save_obj(male, 'male')
save_obj(female, 'female')

In [112]:
male = healthyData

for x in set(male.index.tolist()):
    if x not in gender['male']:
        male = male.drop(labels=x)
        
female = healthyData

for x in set(female.index.tolist()):
    if x not in gender['female']:
        female = female.drop(labels=x)

In [116]:
from sklearn.metrics.pairwise import cosine_similarity

similar = cosine_similarity(prob, male.values)
[x[1] for x in sorted(zip(similar[0], male.index.tolist()), reverse=True)[:3]]

['tyler oakley', 'tom odell', 'ryan gosling']

In [34]:
# load json and create model
optimizer = Adam(lr=1e-5)
loss = 'binary_crossentropy'
metrics = ['mse', 'accuracy']

json_file = open('./Architecture/modelCat2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model2 = model_from_json(loaded_model_json)
# load weights into new model
loaded_model2.load_weights("./Weights/ModelCropCat2/weights-09-0.87.hdf5")
print("Loaded model from disk")
loaded_model2.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Loaded model from disk


In [7]:
# load json and create model
optimizer = Adam(lr=1e-5)
loss = {'dense_3': 'mean_squared_error', 'dense_4': 'binary_crossentropy'}
metrics = ['mse', 'accuracy']

json_file = open('./Architecture/VGGOutputs.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model3 = model_from_json(loaded_model_json)
# load weights into new model
loaded_model3.load_weights("./Weights/VGGOutputs/weights-11-0.31.hdf5")
print("Loaded model from disk")
loaded_model3.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Loaded model from disk


In [1]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [6]:
def generator(indices, batchSize = 128):

    while True:

        for i in range(0,len(indices), batchSize):
            X = []
            Y = []
            Z = []
            batch_indices = indices[i:i+batchSize]

            for x in batch_indices:
                X.append(np.array(Image.open('./Data2/' + x))/255)
                Y.append(labels[x])
                Z.append(labels2[x])

            X = np.array(X)
            Y = np.array(Y)
            Z = np.array(Z)
            yield (X, {'dense_3':Y, 'dense_4':Z})

In [21]:
batchSize = 1
labels = load_obj('labelsReal2')
labels2 = load_obj('labelsCat2')
partition = {}
partition['train'] = load_obj('./Pickles/train')
partition['test'] = load_obj('./Pickles/test')
generator_test = generator(partition['test'], batchSize)

NameError: name 'generator' is not defined

In [8]:
trueDict = {}

for key in partition['test']:
    trueDict[key] = [i for i,x in enumerate(labels2[key]) if x == 1]

In [9]:
yTrue = []
for key in partition['test']:
    yTrue += trueDict[key]

In [10]:
yPred = load_obj('predictions')

In [53]:
len(yTrue)

226268

In [28]:
[precision_score(yTrue, yPred, average='macro', labels=[i]) for i in range(25)]

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.5055023303987571,
 0.31982421875,
 0.0,
 1.0,
 0.5898962932539276,
 0.5348837209302325,
 0.0,
 0.4179983179142136,
 0.4744219445641247,
 0.0,
 0.34281650071123754,
 0.8409090909090909,
 0.0,
 0.486784140969163,
 0.5976954894622809,
 0.5796660898844876,
 0.0,
 1.0,
 1.0,
 0.45816409423233145,
 1.0,
 0.4017725258493353,
 0.0,
 0.34615384615384615,
 0.7434868585251598]

In [50]:
def generator(indices, batchSize = 128, training=True):

    while True:
        if training:
            np.random.shuffle(indices)

        for i in range(0,len(indices), batchSize):
            X = []
            real = []
            build = []
            eye = []
            hair = []
            race = []
            batch_indices = indices[i:i+batchSize]
            if training:
                batch_indices.sort()
            print(indices)

            for x in batch_indices:
                X.append(np.array(Image.open('./Data2/' + x))/255)
                real.append(labels[x])
                build.append(labels2[x][:4])
                eye.append(labels2[x][4:10])
                hair.append(labels2[x][10:16])
                race.append(labels2[x][16:])

            X = np.array(X)
            real = np.array(real)
            build = np.array(build)
            eye = np.array(eye)
            hair = np.array(hair)
            race = np.array(race)
            yield (X, {'real':real, 'build':build, 'eye':eye, 'hair':hair, 'race':race})

In [41]:
batchSize=2

In [51]:
probabilities = finalModel.predict_generator(generator=generator(partition['test'], batchSize, False), steps=5, verbose=1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1/5 [=====>........................] - ETA: 5s['janelle monae183.jpg', 'chris evans225.jpg', 'ray stevenson202.jpg', 'chanel iman39.jpg', 'sushmita sen29.jpg', 'charlotte riley249.jpg', 'janel parrish92.jpg', 'toni garrn366.jpg', 'jennifer love hewitt143.jpg', 'paul rudd18.jpg', 'julie benz256.jpg', 'hilary swank131.jpg', 'quentin tarantino355.jpg', 'cristiano ronaldo259.jpg', 'kelly osbourne187.jpg', 'lionel messi128.jpg', 'beyonce knowles445.jpg', 'alicia vikander297.jpg', 'aaron taylor johnson217.jpg', 'emilia clarke145.jpg', 'kelly osbourne144.jpg', 'brenda song215.jpg', 'jessica szohr162.jpg', 'roberto firmino164.jpg', 'caroline trentini116.jpg', 'brendon urie212.jpg', 'denzel washington465.jpg', 'michelle pfeiffer131.jpg', 'ian somerhalder417.jpg', 'bethenny frankel317.jpg', 'ruth negga174.jpg', 'padma lakshmi298.jpg', 'dinah jane hansen371.jpg', 'zinedine zidane285.jpg', 'stephen amell408.jpg', 'troian bellisario173.jpg', 'blac chyna101.jpg', 'angelique kerber193.jpg', 'uzo adub

2/5 [===========>..................] - ETA: 3s['janelle monae183.jpg', 'chris evans225.jpg', 'ray stevenson202.jpg', 'chanel iman39.jpg', 'sushmita sen29.jpg', 'charlotte riley249.jpg', 'janel parrish92.jpg', 'toni garrn366.jpg', 'jennifer love hewitt143.jpg', 'paul rudd18.jpg', 'julie benz256.jpg', 'hilary swank131.jpg', 'quentin tarantino355.jpg', 'cristiano ronaldo259.jpg', 'kelly osbourne187.jpg', 'lionel messi128.jpg', 'beyonce knowles445.jpg', 'alicia vikander297.jpg', 'aaron taylor johnson217.jpg', 'emilia clarke145.jpg', 'kelly osbourne144.jpg', 'brenda song215.jpg', 'jessica szohr162.jpg', 'roberto firmino164.jpg', 'caroline trentini116.jpg', 'brendon urie212.jpg', 'denzel washington465.jpg', 'michelle pfeiffer131.jpg', 'ian somerhalder417.jpg', 'bethenny frankel317.jpg', 'ruth negga174.jpg', 'padma lakshmi298.jpg', 'dinah jane hansen371.jpg', 'zinedine zidane285.jpg', 'stephen amell408.jpg', 'troian bellisario173.jpg', 'blac chyna101.jpg', 'angelique kerber193.jpg', 'uzo adub

3/5 [=================>............] - ETA: 2s

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4/5 [=======================>......] - ETA: 1s['janelle monae183.jpg', 'chris evans225.jpg', 'ray stevenson202.jpg', 'chanel iman39.jpg', 'sushmita sen29.jpg', 'charlotte riley249.jpg', 'janel parrish92.jpg', 'toni garrn366.jpg', 'jennifer love hewitt143.jpg', 'paul rudd18.jpg', 'julie benz256.jpg', 'hilary swank131.jpg', 'quentin tarantino355.jpg', 'cristiano ronaldo259.jpg', 'kelly osbourne187.jpg', 'lionel messi128.jpg', 'beyonce knowles445.jpg', 'alicia vikander297.jpg', 'aaron taylor johnson217.jpg', 'emilia clarke145.jpg', 'kelly osbourne144.jpg', 'brenda song215.jpg', 'jessica szohr162.jpg', 'roberto firmino164.jpg', 'caroline trentini116.jpg', 'brendon urie212.jpg', 'denzel washington465.jpg', 'michelle pfeiffer131.jpg', 'ian somerhalder417.jpg', 'bethenny frankel317.jpg', 'ruth negga174.jpg', 'padma lakshmi298.jpg', 'dinah jane hansen371.jpg', 'zinedine zidane285.jpg', 'stephen amell408.jpg', 'troian bellisario173.jpg', 'blac chyna101.jpg', 'angelique kerber193.jpg', 'uzo adub

5/5 [==============================] - 6s 1s/step


In [46]:
probabilities[1].shape

(10, 4)

In [103]:
buildTrue = []
eyeTrue = []
hairTrue = []
raceTrue = []

for key in partition['test']:
    classes = labels2[key]
    buildTrue.append(classes[:4])
    eyeTrue.append(classes[4:10])
    hairTrue.append(classes[10:16])
    raceTrue.append(classes[16:])
    
buildTrue = [list(x).index(max(x)) for x in buildTrue]
eyeTrue = [list(x).index(max(x)) for x in eyeTrue]
hairTrue = [list(x).index(max(x)) for x in hairTrue]
raceTrue = [list(x).index(max(x)) for x in raceTrue]

In [111]:
precision_score(buildTrue[:10], [list(x).index(max(x)) for x in probabilities[1]], average='macro')

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.03571428571428571

In [78]:
average_precision_score(np.array(eye).reshape((-1)), probabilities[2].reshape((-1)), average='macro')

0.22899487452119027

In [79]:
average_precision_score(np.array(hair).reshape((-1)), probabilities[3].reshape((-1)), average='macro')

0.42980311355311357

In [80]:
average_precision_score(np.array(race).reshape((-1)), probabilities[4].reshape((-1)), average='macro')

0.5236657786657787

In [81]:
average_precision_score(np.array(eye).reshape((-1)), probabilities[2].reshape((-1)), average='micro')

0.22899487452119027

In [82]:
average_precision_score(np.array(hair).reshape((-1)), probabilities[3].reshape((-1)), average='micro')

0.42980311355311357

In [83]:
average_precision_score(np.array(race).reshape((-1)), probabilities[4].reshape((-1)), average='micro')

0.5236657786657787

In [77]:
probabilities[1].reshape((-1)).shape

(40,)

In [70]:
np.array(build).reshape((-1)).shape

(40,)

In [96]:
probabilities[1]

array([[0.18641634, 0.4212414 , 0.3198366 , 0.07250571],
       [0.20000604, 0.34876838, 0.35753086, 0.09369466],
       [0.22772262, 0.40446484, 0.23227608, 0.13553655],
       [0.14805113, 0.3325408 , 0.40589797, 0.11351015],
       [0.24198188, 0.37623817, 0.23454534, 0.14723462],
       [0.21558978, 0.38212827, 0.29085052, 0.11143145],
       [0.18523741, 0.44986093, 0.25511715, 0.10978451],
       [0.18772002, 0.2687444 , 0.38230193, 0.16123368],
       [0.19517024, 0.3877447 , 0.2758322 , 0.14125292],
       [0.1566322 , 0.39508718, 0.33805743, 0.11022326]], dtype=float32)

In [95]:
[list(x).index(max(x)) for x in probabilities[1]]

[1, 2, 1, 2, 1, 1, 1, 2, 1, 1]

In [112]:
buildTrue = []
eyeTrue = []
hairTrue = []
raceTrue = []

for key in partition['test']:
    classes = labels2[key]
    buildTrue.append(classes[:4])
    eyeTrue.append(classes[4:10])
    hairTrue.append(classes[10:16])
    raceTrue.append(classes[16:])
    
buildTrue = [list(x).index(max(x)) for x in buildTrue]
eyeTrue = [list(x).index(max(x)) for x in eyeTrue]
hairTrue = [list(x).index(max(x)) for x in hairTrue]
raceTrue = [list(x).index(max(x)) for x in raceTrue]

scores = {}

for x in os.listdir('./Weights/VGG/'):
    print(x)
    model = model_from_json(loaded_model_json)
    model.load_weights('./Weights/VGG/'+x)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    probabilities = model.predict_generator(generator=generator(partition['test'], batchSize, False), steps=5, verbose=1)
    ap1 = precision_score(buildTrue[:10], [list(x).index(max(x)) for x in probabilities[1]], average='macro')
    ap2 = precision_score(eyeTrue[:10], [list(x).index(max(x)) for x in probabilities[2]], average='macro')
    ap3 = precision_score(hairTrue[:10], [list(x).index(max(x)) for x in probabilities[3]], average='macro')
    ap4 = precision_score(raceTrue[:10], [list(x).index(max(x)) for x in probabilities[4]], average='macro')
    mean = (ap1 + ap2 + ap3 + ap4) / 4
    print(ap1, ap2, ap3, ap4, mean)
    scores[x] = {'build':ap1, 'eye':ap2, 'hair':ap3, 'race':ap4, 'map':mean}
    save_obj(scores)

KeyboardInterrupt: 

In [113]:
socres = load_obj('weights-19-5.18.hdf5scores')

In [116]:
socres.keys()

dict_keys(['weights-19-5.18.hdf5'])